In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

In [ ]:
data.info()

In [ ]:
data

In [ ]:
data.isnull().count()

In [ ]:
data=data.drop(['MiscFeature'],axis=1)

In [ ]:
data.info()

In [ ]:
data['SalePrice'].describe()

In [ ]:
import seaborn as sns

In [ ]:

sns.distplot(data['SalePrice'])

In [ ]:

data['SalePrice'].skew()

In [ ]:
data['SalePrice'].kurt()

In [ ]:
data.columns

In [ ]:
#scatter plot grlivarea/saleprice
var = 'GrLivArea'
data_df = pd.concat([data['SalePrice'], data[var]], axis=1)
data_df.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

In [ ]:
var='TotalBsmtSF'
data_df=pd.concat([data['SalePrice'],data[var]],axis=1)
data_df.plot.scatter(x=var,y='SalePrice',ylim=(0,800000))

In [ ]:



import matplotlib.pyplot as plt

In [ ]:
var='OverallQual'
data_df=pd.concat([data['SalePrice'],data[var]],axis=1)
f,ax=plt.subplots(figsize=(8,6))
fig=sns.boxplot(x=var,y='SalePrice',data=data_df)
fig.axis(ymin=0,ymax=800000)

In [ ]:
var = 'YearBuilt'
data_df = pd.concat([data['SalePrice'], data[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="SalePrice", data=data_df)
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90);


we can conclude that:

'GrLivArea' and 'TotalBsmtSF' seem to be linearly related with 'SalePrice'. Both relationships are positive, which means that as one variable increases, the other also increases. In the case of 'TotalBsmtSF', we can see that the slope of the linear relationship is particularly high.
'OverallQual' and 'YearBuilt' also seem to be related with 'SalePrice'. The relationship seems to be stronger in the case of 'OverallQual', where the box plot shows how sales prices increase with the overall quality.

In [ ]:
corrmat=data.corr()

In [ ]:
f,ax=plt.subplots(figsize=(12,9))
sns.heatmap(corrmat,vmax=0.8,square=True)

At first sight, there are two red colored squares that get my attention. The first one refers to the 'TotalBsmtSF' and '1stFlrSF' variables, and the second one refers to the 'GarageX' variables. Both cases show how significant the correlation is between these variables. Actually, this correlation is so strong that it can indicate a situation of multicollinearity. If we think about these variables, we can conclude that they give almost the same information so multicollinearity really occurs. Heatmaps are great to detect this kind of situations and in problems dominated by feature selection, like ours, they are an essential tool.

Another thing that got my attention was the 'SalePrice' correlations. We can see our well-known 'GrLivArea', 'TotalBsmtSF', and 'OverallQual' saying a big 'Hi!', but we can also see many other variables that should be taken into account. That's what we will do next.



In [ ]:

import numpy as np
k=10
cols=corrmat.nlargest(k,'SalePrice')['SalePrice'].index
cm=np.corrcoef(data[cols].values.T)
sns.set(font_scale=2.0)
hm=sns.heatmap(cm,cbar=True,annot=True,fmt='.2f',annot_kws={'size':10},yticklabels=cols.values,xticklabels=cols.values)
plt.show()

OverallQual', 'GrLivArea' and 'TotalBsmtSF' are strongly correlated with 'SalePrice'. Check!
'GarageCars' and 'GarageArea' are also some of the most strongly correlated variables. However, as we discussed in the last sub-point, the number of cars that fit into the garage is a consequence of the garage area. 'GarageCars' and 'GarageArea' are like twin brothers. You'll never be able to distinguish them. Therefore, we just need one of these variables in our analysis (we can keep 'GarageCars' since its correlation with 'SalePrice' is higher).
'TotalBsmtSF' and '1stFloor' also seem to be twin brothers. We can keep 'TotalBsmtSF' just to say that our first guess was right (re-read 'So... What can we expect?').
'FullBath'?? Really?
'TotRmsAbvGrd' and 'GrLivArea', twin brothers again. Is this dataset from Chernobyl?
Ah... 'YearBuilt'... It seems that 'YearBuilt' is slightly correlated with 'SalePrice'. Honestly, it scares me to think about 'YearBuilt' because I start feeling that we should do a little bit of time-series analysis to get this right. I'll leave this as a homework for you.

In [ ]:
sns.set()
cols=['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(data[cols],height=5)
plt.show()

Although we already know some of the main figures, this mega scatter plot gives us a reasonable idea about variables relationships.

One of the figures we may find interesting is the one between 'TotalBsmtSF' and 'GrLiveArea'. In this figure we can see the dots drawing a linear line, which almost acts like a border. It totally makes sense that the majority of the dots stay below that line. Basement areas can be equal to the above ground living area, but it is not expected a basement area bigger than the above ground living area (unless you're trying to buy a bunker).

The plot concerning 'SalePrice' and 'YearBuilt' can also make us think. In the bottom of the 'dots cloud', we see what almost appears to be a shy exponential function (be creative). We can also see this same tendency in the upper limit of the 'dots cloud' (be even more creative). Also, notice how the set of dots regarding the last years tend to stay above this limit (I just wanted to say that prices are increasing faster now).

In [ ]:
total=data.isnull().sum().sort_values(ascending=False)
percent=((data.isnull().sum()/len(data))*100).sort_values(ascending=False)
missing_data=pd.concat([total,percent],axis=1,keys=['Total','percent'])
missing_data.head()

In [ ]:
missing_data.head(20)

Imputing missing values



We impute them by proceeding sequentially through features with missing values

PoolQC : data description says NA means "No Pool". That make sense, given the huge ratio of missing value (+99%) and majority of houses have no Pool at all in general.


In [ ]:
data['PoolQC']=data['PoolQC'].fillna('None')

Alley : data description says NA means "no Alley"

In [ ]:
data['Alley']=data['Alley'].fillna('None')

In [ ]:
data['Fence']=data['Fence'].fillna('None')

In [ ]:
data['FireplaceQu']=data['FireplaceQu'].fillna('None')

In [ ]:
data['LotFrontage']=data.groupby('Neighborhood')['LotFrontage'].transform(lambda x:x.fillna(x.median()))

In [ ]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    data[col]=data[col].fillna('None')

In [ ]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    data[col]=data[col].fillna(0)

In [ ]:
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    data[col]=data[col].fillna(0)

In [ ]:
data['MasVnrtype']=data['MasVnrType'].fillna('None')

In [ ]:
data['MasVnrArea']=data['MasVnrArea'].fillna(0)

In [ ]:
data.shape

In [ ]:
data.drop('Id',axis=1,inplace=True)

In [ ]:
data.shape
data.isnull().sum()
data.info()

In [ ]:
data['MSZoning']=data['MSZoning'].fillna(data['MSZoning'].mode()[0])

In [ ]:
data=data.drop(['Utilities'],axis=1)

In [ ]:
data.shape

In [ ]:
data['Functional']=data['Functional'].fillna('typ')

In [ ]:
data['Electrical']=data['Electrical'].fillna(data['Electrical'].mode()[0])

In [ ]:
data['KitchenQual']=data['KitchenQual'].fillna(data['KitchenQual'].mode()[0])

In [ ]:
data['Exterior1st']=data['Exterior1st'].fillna(data['Exterior1st'].mode()[0])

In [ ]:
data['Exterior2nd']=data['Exterior2nd'].fillna(data['Exterior2nd'].mode()[0])

In [ ]:
data['SaleType']=data['SaleType'].fillna(data['SaleType'].mode()[0])

In [ ]:
data['MSSubClass']=data['MSSubClass'].fillna('None')

In [ ]:
data_na=data.isnull().sum()
missing_ratio=(data_na/len(data)*100)
missing_ratio=missing_ratio.drop(missing_ratio[missing_ratio==0].index)
missing_data=pd.DataFrame(missing_ratio)
missing_data



In [ ]:
data['MasVnrType'].isnull().count()
data['BsmtQual'].isnull().count()
data['BsmtExposure'].isnull().count()
data['BsmtFinType1'].isnull().count()
data['BsmtFinType2'].isnull().count()

In [ ]:
fig,ax=plt.subplots()
ax.scatter(x=data['GrLivArea'],y=data['SalePrice'])
plt.ylabel('SalePrice')
plt.xlabel('GrLivArea')
plt.show()

We can see at the bottom right two with extremely large GrLivArea that are of a low price. These values are huge oultliers. Therefore, we can safely delete them.[](http://)

In [ ]:
data=data.drop(data[(data['GrLivArea']>4000) & (data['SalePrice']<300000)].index)

SalePrice is the variable we need to predict. So let's do some analysis on this variable first.

In [ ]:
from scipy import stats
from scipy.stats import norm, skew 

In [ ]:
sns.distplot(data['SalePrice'],fit=norm)
mu,sigma=norm.fit(data['SalePrice'])

The target variable is right skewed. As (linear) models love normally distributed data , we need to transform this variable and make it more normally distributed.

In [ ]:


data['SalePrice']=np.log1p(data['SalePrice'])
sns.distplot(data['SalePrice'],fit=norm)
mu,sigma=norm.fit(data['SalePrice'])
mu,sigma

In [ ]:


data.info()

In [ ]:
data['MSSubClass']=data['MSSubClass'].apply(str)
data['OverallCond']=data['OverallCond'].astype(str)
data['YrSold']=data['YrSold'].astype(str)
data['MoSold']=data['MoSold'].astype(str)

Now using Label encoder for categorical features

In [ ]:
from sklearn.preprocessing import LabelEncoder
cols=[ 
        'ExterQual', 'ExterCond','HeatingQC',  'KitchenQual',  
         'Functional',   'LandSlope',
        'LotShape', 'PavedDrive', 'Street',  'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold']
for c in cols:
    lbl=LabelEncoder()
    lbl.fit(data[c].values)
    data[c]=lbl.transform(data[c].values)
    
    

In [ ]:
data.shape

In [ ]:
numeric_feat=data.dtypes[data.dtypes != 'object'].index

In [ ]:
skewed_feats=data[numeric_feat].apply(lambda x: skew(x.dropna()))

In [ ]:
skewness=pd.DataFrame(skewed_feats)

In [ ]:
abs(skewness)

In [ ]:
skewness=skewness[abs(skewness)>0.75]

In [ ]:
skewness

In [ ]:
skewed_features=skewness.index

In [ ]:
skewed_features

In [ ]:
from scipy.special import boxcox1p
data['GrLivArea']

In [ ]:
lam=0.15

In [ ]:
for feat in skewed_features:
    data[feat]=boxcox1p(data[feat],lam)

In [ ]:
data=pd.get_dummies(data)

In [ ]:
data.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
sale_price=data['SalePrice']

In [ ]:
data.drop('SalePrice',axis=1)

In [ ]:
data_train,data_test,sale_price_train,sale_price_test=train_test_split(data,sale_price,test_size=0.2)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr=LinearRegression()

In [ ]:
lr.fit(data_train,sale_price_train)

In [ ]:
pred=lr.predict(data_test)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(sale_price_test,pred)

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
la=Lasso()

In [ ]:
la.fit(data_train,sale_price_train)

In [ ]:
pred=la.predict(data_test)

In [ ]:
mean_squared_error(sale_price_test,pred)

In [ ]:
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor

In [ ]:
rfr=RandomForestRegressor()

In [ ]:
rfr.fit(data_train,sale_price_train)

In [ ]:
pred=rfr.predict(data_test)

In [ ]:
mean_squared_error(sale_price_test,pred)

In [ ]:
gbr=GradientBoostingRegressor()

In [ ]:
gbr.fit(data_train,sale_price_train)

In [ ]:
pred=gbr.predict(data_test)

In [ ]:
mean_squared_error(pred,sale_price_test)

In [ ]:
from sklearn.model_selection import cross_val_score,KFold
#validationfunction
folds=5
def rmse_cv(model):
    kf=KFold(folds,shuffle=True,random_state=42).get_n_splits(data_train.values)
    rmse=np.sqrt(-cross_val_score(model,data_train.values,sale_price_train,scoring='neg_mean_squared_error',cv=kf))
    return rmse

In [ ]:
gbr=GradientBoostingRegressor(n_estimators=3000,learning_rate=0.05,max_depth=4,random_state=4)

In [ ]:
gbr.fit(data_train,sale_price_train)

In [ ]:
pred=gbr.predict(data_test)

In [ ]:
mean_squared_error(sale_price_test,pred)

In [ ]:
score=rmse_cv(gbr)

In [ ]:
score.mean()

In [ ]:
import xgboost as xgb

In [ ]:
model_xgb=xgb.XGBRegressor(n_estimators=2200,max_depth=3)

d

In [ ]:
score=rmse_cv(model_xgb)

score=rmse_cv(model_xgb)

In [ ]:
score.mean()

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler


In [ ]:
Enet=make_pipeline(RobustScaler(),ElasticNet())

In [ ]:
score=rmse_cv(Enet)

In [ ]:
score.mean()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self,models):
        self.models=models
    
    def fit(self,X,y):
        self.models_=[clone(x) for x in self.models]
        
        for model in self.models_:
            model.fit(X,y)
        return self
    
    def predict(self,X):
        predictions=np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions,axis=1)

In [ ]:
avg_models=AveragingModels(models=(la,Enet,gbr,model_xgb))

In [ ]:
score = rmse_cv(avg_models)

In [ ]:
score.mean()